In [1]:
import sys
!{sys.executable} -m pip install SQLAlchemy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


# Proyecto SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

**Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.**

---

In [2]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = """
    SELECT *
    FROM books
    LIMIT 10
"""

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


## Número de libros publicados después del 1 de enero de 2000

In [4]:
query2 = """
    SELECT COUNT(*) AS books_after_2000
    FROM books
    WHERE publication_date > '2000-01-01';
"""

pd.io.sql.read_sql(query2, con = engine)

,books_after_2000
0,819


### **Conclusión:**

Hay 819 libros que fueron publicados despues del 2000-01-01.

---

## Número de reseñas de usuarios y la calificación promedio para cada libro

In [5]:
query3 = """
   SELECT
    b.title,
    COUNT(DISTINCT r.review_id) AS total_reviews,
    ROUND(AVG(rt.rating), 2) AS average_rating
FROM reviews r
JOIN books b ON r.book_id = b.book_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.title
ORDER BY total_reviews DESC;
"""

pd.io.sql.read_sql(query3, con = engine)

,title,total_reviews,average_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,The Da Vinci Code (Robert Langdon #2),6,3.83
3,The Road,6,3.77
4,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
...,...,...,...
988,Naked Empire (Sword of Truth #8),1,3.50
989,Moo Baa La La La!,1,3.00
990,Merrick (The Vampire Chronicles #7),1,4.00
991,Babyville,1,3.50


### **Conclusion:**

Para responder a esa pregunta necesitamos combinar las tablas reviews y ratings, ambas vinculadas a books a través de book_id.

**La calificción promedio de los libros suele ir desde 3.00 - 4.41, podemos ver que algunos libros que tuvieron menos reseñas y aún asi tuvieron una clificación promedio mas alta.**

---

### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [6]:
query4 = """
   SELECT 
    p.publisher, 
    COUNT(b.book_id) AS total_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_books DESC
LIMIT 5;
"""

pd.io.sql.read_sql(query4, con = engine)

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


### **Conclusión:**

La editorial que ha publicado el mayor número de libros con más de 50 páginas es **Penguin Books** con **42 libros.**

**Qué hace cada parte?**

- **b.num_pages > 50:** Filtra solo libros con más de 50 páginas.

- **JOIN publishers:** Para obtener el nombre de la editorial.

- **GROUP BY p.publisher:** Agrupa por editorial.

- **COUNT(b.book_id):** Cuenta los libros por editorial.

- **ORDER BY ... LIMIT 1:** Nos da la editorial con más libros publicados de más de 50 páginas.

---

### **Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.**

**1.** Unimos books, authors y ratings.

**2.** Agrupamos por libro para calcular:

    - el número de calificaciones por libro,

    - la calificación promedio de cada libro.

**3.** Filtramos para quedarnos solo con los libros que tienen al menos 50 calificaciones.

**4.** Calculamos el promedio de calificaciones por autor (solo de libros válidos).

**5.** Mostramos el autor con el mayor promedio.

In [7]:
query5 = """
   WITH book_stats AS (
    SELECT 
        b.book_id,
        b.author_id,
        COUNT(rt.rating_id) AS rating_count,
        AVG(rt.rating) AS avg_rating
    FROM books b
    JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(rt.rating_id) >= 50
)

SELECT 
    a.author,
    ROUND(AVG(bs.avg_rating), 2) AS author_avg_rating
FROM book_stats bs
JOIN authors a ON bs.author_id = a.author_id
GROUP BY a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
"""

pd.io.sql.read_sql(query5, con = engine)

,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


### **Conlcusión:**

El autor con la calificación promedio del libro es **J.K. Rowling/Mary GrandPré** con un promedio de **4.28.**

**Que hace cada parte:**

- WITH book_stats AS (...): Calcula cuántas calificaciones y qué promedio tiene cada libro.

- HAVING COUNT(rt.rating_id) >= 50: Filtra libros con al menos 50 calificaciones.

- Luego, se agrupan los resultados por autor para calcular el promedio de sus libros "válidos".

- LIMIT 1: Devuelve solo el autor con mejor promedio.

---

## Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

**Esta consulta requiere dos pasos principales:**

- Encontrar los usuarios que han hecho más de 50 calificaciones **(en la tabla ratings).**

- Calcular el promedio de reseñas escritas **(en la tabla reviews)**  por esos usuarios.

In [8]:
query6 = """
   SELECT 
    ROUND(AVG(review_count), 2) AS avg_reviews_per_user
FROM (
    SELECT 
        r.username,
        COUNT(rv.review_id) AS review_count
    FROM ratings r
    JOIN reviews rv ON r.username = rv.username
    GROUP BY r.username
    HAVING COUNT(r.rating_id) > 50
) AS subquery;
"""

pd.io.sql.read_sql(query6, con = engine)

,avg_reviews_per_user
0,719.76


- Primero, agrupamos por username y contamos cuántas reseñas de texto ha hecho cada uno.

- Limitamos solo a quienes han hecho más de 50 calificaciones.

- Finalmente, calculamos el promedio de reseñas entre esos usuarios.

**Conclusión:**

El **número promedio de reseñas** de texto entre los usuarios que calificaron más de 50 libros es **719.76**

---